# 5. LSH-Louvain
This notebooks implements a hybrid approach combining LSH and Louvain, so that that tha advantages of each method can be maximized while minimizing their limitations.

In [136]:
# Imports

import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np
from collections import defaultdict
from sklearn.metrics.pairwise import cosine_similarity
import re
import time
import tracemalloc

import networkx as nx
import community as community_louvain
from sklearn.metrics import davies_bouldin_score
import pickle


# Load precomputed dataframe + embeddings
df = pd.read_pickle("games_df.pkl")
embeddings = np.load("embeddings.npy")

print("Embeddings shape:", embeddings.shape)
print("Number of games:", len(df))


Embeddings shape: (10476, 384)
Number of games: 10476


## generate random hyperplanes

In [137]:
def generate_hyperplanes(num_tables, num_planes, dim, seed= 0):
    rng = np.random.RandomState(seed)
    hyperplanes = []
    for _ in range(num_tables):
        planes = rng.randn(num_planes, dim)
        planes /= np.linalg.norm(planes, axis = 1, keepdims = True) +1e-9
        hyperplanes.append(planes)
    
    return hyperplanes

# check it
dim = embeddings.shape[1]
num_tables = 10
num_planes = 16

hyperplanes = generate_hyperplanes(num_tables, num_planes, dim, seed=33)
print(hyperplanes)

[array([[-0.01636433, -0.0822688 , -0.07879106, ..., -0.03037444,
         0.02857754,  0.05055213],
       [ 0.04226748,  0.01131865, -0.00684458, ..., -0.00090081,
        -0.04132469,  0.03906224],
       [ 0.00036863, -0.00614149, -0.0029053 , ..., -0.02039806,
        -0.06660375, -0.00144424],
       ...,
       [-0.02160066,  0.00627249,  0.05156927, ...,  0.04815518,
         0.03370179, -0.03259623],
       [-0.16762874,  0.01323529,  0.035048  , ...,  0.07726289,
         0.00270281,  0.01887253],
       [ 0.00026104,  0.00025794,  0.05709854, ..., -0.07947919,
         0.00487741,  0.00843518]], shape=(16, 384)), array([[ 0.01588397,  0.08047979, -0.00158552, ...,  0.00705402,
         0.04997843,  0.02041587],
       [ 0.01394603, -0.08109805, -0.02365737, ..., -0.12642717,
        -0.02261045,  0.05341474],
       [-0.05057007,  0.06450797,  0.05145814, ..., -0.04600032,
         0.03888609, -0.01980033],
       ...,
       [-0.02351672, -0.00966409, -0.03954437, ...,  0.0

## hash each embedding into buckets 

In [138]:
def hash_v(v, planes):
    projections = planes @ v
    bits = projections > 0
    key = ''.join('1' if b else '0' for b in bits)
    return key


## Get LSH index, which is tables with buckets

In [139]:
# Get LSH index, which is tables with buckets

def build_lsh_idx(embeddings, hyperplanes):
    n, dim = embeddings.shape
    num_tables = len(hyperplanes)

    tables = [defaultdict(set) for _ in range(num_tables)]

    for idx, v in enumerate(embeddings):
        for t, planes in enumerate(hyperplanes):
            key = hash_v(v, planes)
            tables[t][key].add(idx)
    
    return tables

# build it
tables = build_lsh_idx(embeddings, hyperplanes)
print("Built LSH tables.")


Built LSH tables.


## get candidate neighbours for a game

In [140]:
def lsh_query(idx, embeddings, hyperplanes, tables):
    v = embeddings[idx]
    candidates = set()

    for t, planes in enumerate(hyperplanes):
        key = hash_v(v, planes)
        bucket = tables[t].get(key, set())
        candidates |= bucket

    candidates.discard(idx)
    return candidates

# quick test
cand = lsh_query(0, embeddings, hyperplanes, tables)
print("Candidates for game 0:", len(cand))
for candidate in list(cand)[:10]:
    print(df.loc[candidate, 'name'])


Candidates for game 0: 23
Trine 2: Goblin Menace
Rake Remastered
Total War: Shogun 2 - Fall of the Samurai: The Saga Faction Pack
DNF Duel: Season Pass
Lords of the Fallen: The Foundation Boost
Tri.Defender
Injustice 2: Reverse Flash
Deep Space Scoundrel
Quake III: Team Arena
Running into the Cyberpunk


## LSH Recommender (from section 3.2.2)

In [141]:
# Rank candidates by cosine similarity + rating + recency
def recommend_lsh_embedding(idx, embeddings, hyperplanes, tables, df, top=5):
    v = embeddings[idx]
    cand = lsh_query(idx, embeddings, hyperplanes, tables)

    if not cand:
        print(f"No candidates for {df.loc[idx, 'name']}")
        return []

    cand_indices = np.array(list(cand), dtype=int)

    sim = cosine_similarity(
        v.reshape(1, -1),
        embeddings[cand_indices]
    )  # shape: (1, n_candidates)

    sim_flat = sim[0]  
    cos_norm = (sim_flat + 1.0) / 2.0

    ratings = df.loc[cand_indices, 'steam_rating'].astype(float).to_numpy()
    recency = df.loc[cand_indices, 'recency'].astype(float).to_numpy()

    w_sim = 0.7
    w_rating = 0.2
    w_date = 0.1

    final_score = (w_sim*cos_norm + w_rating*ratings + w_date*recency)
    
    order = np.argsort(final_score)[::-1]
    ordered_indices = cand_indices[order]
    ordered_cos_norm = cos_norm[order]    
    ordered_rating = ratings[order]
    ordered_recency = recency[order]
    ordered_score = final_score[order]

    results = list(zip(
        ordered_indices[:top], 
        ordered_cos_norm[:top],
        ordered_rating[:top],
        ordered_recency[:top],
        ordered_score[:top]))

    print(f"Query: {df.loc[idx, 'name']}")
    print("Recommendations (cosine_norm, rating, recency, score):")

    for j, cos_s, r, rec, score in results:
        print(
            f" --> {df.loc[j, 'name']} "
            f"(cos={cos_s:.3f}, rating={r:.3f}, recency = {rec:.3f}, score={score:.3f})"
        )

    return results

# test LSH-only recommender
recommend_lsh_embedding(0, embeddings, hyperplanes, tables, df, top=3)


Query: Hunter Hitman
Recommendations (cosine_norm, rating, recency, score):
 --> Running into the Cyberpunk (cos=0.677, rating=1.000, recency = 0.941, score=0.768)
 --> Scratch Man (cos=0.681, rating=0.923, recency = 0.955, score=0.757)
 --> Freebooter of Splorr!! (cos=0.654, rating=1.000, recency = 0.919, score=0.750)


[(np.int64(10144),
  np.float32(0.6766021),
  np.float64(1.0),
  np.float64(0.9410124943999204),
  np.float64(0.7677227370574846)),
 (np.int64(6376),
  np.float32(0.6806127),
  np.float64(0.9230769231),
  np.float64(0.9554482552640748),
  np.float64(0.7565890765328211)),
 (np.int64(9205),
  np.float32(0.6544188),
  np.float64(1.0),
  np.float64(0.9187117327890886),
  np.float64(0.7499643396302271))]

### Cosine similarity matrix 

In [142]:
t0_sim = time.perf_counter()
sim_matrix = cosine_similarity(embeddings)
print("Cosine similarity matrix computed in:", time.perf_counter() - t0_sim, "sec")


Cosine similarity matrix computed in: 0.7867610930006776 sec


## Build a similarity graph using LSH candidates

In [143]:
def build_graph_from_lsh(embeddings, hyperplanes, tables, sim_matrix, threshold=0.8):
    """
    Build an undirected graph:

    - Node: game index
    - Edge (i, j): if j is in LSH candidates of i and sim_matrix[i, j] >= threshold
    - Edge weight: cosine similarity from sim_matrix
    """
    n = embeddings.shape[0]
    G = nx.Graph()
    G.add_nodes_from(range(n))

    seen_edges = set()  # avoid duplicates

    for i in range(n):
        cand = list(lsh_query(i, embeddings, hyperplanes, tables))
        if not cand:
            continue

        cand = np.array(cand, dtype=int)

        # vectorized: all similarities from row i to its candidates
        sims = sim_matrix[i, cand]

        # keep only candidates above threshold
        mask = sims >= threshold
        cand_kept = cand[mask]
        sims_kept = sims[mask]

        for j, sim_ij in zip(cand_kept, sims_kept):
            if j == i:
                continue
            a, b = (i, j) if i < j else (j, i)
            if (a, b) in seen_edges:
                continue

            G.add_edge(int(a), int(b), weight=float(sim_ij))
            seen_edges.add((a, b))

    print(
        f"LSH graph built: nodes={G.number_of_nodes()}, "
        f"edges={G.number_of_edges()}"
    )
    return G

# example: build graph with some threshold
threshold = 0.7
G_lsh = build_graph_from_lsh(embeddings, hyperplanes, tables, sim_matrix, threshold=threshold)


LSH graph built: nodes=10476, edges=2336


## Evaluate Louvain

In [144]:
def evaluate_louvain(
    G: nx.Graph,
    embeddings: np.ndarray,
    resolution: float = 1.0,
    seed: int = 0,
) -> dict:
    """
    Run Louvain on graph G, compute:
    - modularity
    - #clusters, cluster sizes
    - silhouette, Davies-Bouldin on embeddings
    - runtime, peak memory
    Returns a dict of metrics.
    """
    # Measure memory + time
    tracemalloc.start()
    t0_local = time.time()

    partition = community_louvain.best_partition(
        G,
        weight="weight",
        resolution=resolution,
        random_state=seed,
    )

    runtime = time.time() - t0_local
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    peak_mb = peak / (1024 ** 2)

    labels = np.array([partition[i] for i in range(len(partition))])
    n_clusters = len(np.unique(labels))

    cluster_sizes = pd.Series(labels).value_counts()
    largest_cluster = int(cluster_sizes.max())
    smallest_cluster = int(cluster_sizes.min())

    modularity = community_louvain.modularity(partition, G, weight="weight")

    try:
        if n_clusters > 1 and largest_cluster > 1:
            db = davies_bouldin_score(embeddings, labels)
        else:
            db = np.nan
    except Exception as e:
        print("Metric error:", e)
        db = np.nan

    return {
        "resolution": resolution,
        "seed": seed,
        "n_clusters": n_clusters,
        "largest_cluster": largest_cluster,
        "smallest_cluster": smallest_cluster,
        "modularity": modularity,
        "davies_bouldin": db,
        "runtime_sec": runtime,
        "peak_mem_mb": peak_mb,
    }, partition


## # Run Louvain on the LSH-derived graph

In [145]:
resolution = 1.5
seed = 0

metrics_lsh, partition_lsh = evaluate_louvain(G_lsh, embeddings, resolution=resolution, seed=seed)

print("Louvain on LSH-graph metrics:")
for k, v in metrics_lsh.items():
    print(f"{k}: {v}")

# Save LSH-Louvain partition and graph
with open("partition_louvain_lsh.pkl", "wb") as f:
    pickle.dump(partition_lsh, f)

with open("graph_louvain_lsh.pkl", "wb") as f:
    pickle.dump(G_lsh, f)

print("Saved LSH-Louvain partition and graph.")


Louvain on LSH-graph metrics:
resolution: 1.5
seed: 0
n_clusters: 9448
largest_cluster: 82
smallest_cluster: 1
modularity: 0.8663734158237371
davies_bouldin: 0.6354720344927514
runtime_sec: 3.4662363529205322
peak_mem_mb: 12.988666534423828
Saved LSH-Louvain partition and graph.


## Hybrid LSH + Louvain recommender

In [146]:
_lsh_louvain_partition_cache = None

def load_lsh_louvain_partition(path="partition_louvain_lsh.pkl"):
    global _lsh_louvain_partition_cache
    if _lsh_louvain_partition_cache is None:
        print("Loading LSH-Louvain partition...")
        with open(path, "rb") as f:
            _lsh_louvain_partition_cache = pickle.load(f)
    return _lsh_louvain_partition_cache


def recommend_lsh_louvain(idx, embeddings, hyperplanes, tables, df,
                          top=10,
                          w_sim=0.7, w_rating=0.2, w_date=0.1,
                          partition_path="partition_louvain_lsh.pkl"):
    """
    Hybrid LSH + Louvain recommender:

    1. LSH gets fast neighbor candidates.
    2. Filter candidates to those in the same Louvain community.
    3. Rank with combined score of (cosine + rating + recency).
    """
    partition = load_lsh_louvain_partition(partition_path)
    cluster_id = partition[idx]

    # 1) LSH candidates
    cand_all = lsh_query(idx, embeddings, hyperplanes, tables)
    if not cand_all:
        print(f"No LSH candidates for {df.loc[idx, 'name']}")
        return []
    
    print(f"[LSH] Retrieved {len(cand_all)} candidates.")


    # 2) restrict to same Louvain community
    cand_filtered = [j for j in cand_all if partition[j] == cluster_id]
    print(f"[Louvain] Candidates in same community: {len(cand_filtered)} "
      f"(community ID = {cluster_id})")

    if not cand_filtered:
        # fallback: if community filter is too strict, use all LSH candidates
        print("[Fallback] No candidates survived Louvain community filter. "
          "Using all LSH candidates instead.")
        cand_filtered = list(cand_all)
    else:
        print(f"[Hybrid] Using {len(cand_filtered)} hybrid candidates.")

    cand_indices = np.array(cand_filtered, dtype=int)

    # 3) compute similarity + rating + recency score
    v = embeddings[idx]
    sim = cosine_similarity(
        v.reshape(1, -1),
        embeddings[cand_indices]
    )[0]

    cos_norm = (sim + 1.0) / 2.0
    ratings = df.loc[cand_indices, "steam_rating"].astype(float).to_numpy()
    recency = df.loc[cand_indices, "recency"].astype(float).to_numpy()

    final_score = (
        w_sim * cos_norm +
        w_rating * ratings +
        w_date * recency
    )

    order = np.argsort(final_score)[::-1]
    ordered_indices = cand_indices[order]
    ordered_cos = cos_norm[order]
    ordered_rat = ratings[order]
    ordered_rec = recency[order]
    ordered_score = final_score[order]

    results = list(zip(
        ordered_indices[:top],
        ordered_cos[:top],
        ordered_rat[:top],
        ordered_rec[:top],
        ordered_score[:top],
    ))

    print(f"Query: {df.loc[idx, 'name']}")
    print("Recommendations (LSH + Louvain):")
    for j, c, r, rec, s in results:
        print(
            f" --> {df.loc[j, 'name']} "
            f"(cos_norm={c:.3f}, rating={r:.3f}, recency={rec:.3f}, score={s:.3f})"
        )

    return results


## Test LSH-Louvain recommender

In [155]:
recommend_lsh_louvain(100, embeddings, hyperplanes, tables, df, top=5)


[LSH] Retrieved 35 candidates.
[Louvain] Candidates in same community: 0 (community ID = 98)
[Fallback] No candidates survived Louvain community filter. Using all LSH candidates instead.
Query: Master of 4 Swords
Recommendations (LSH + Louvain):
 --> Kannagi Usagi (cos_norm=0.794, rating=0.980, recency=0.961, score=0.848)
 --> Blade Fury (cos_norm=0.765, rating=0.769, recency=0.955, score=0.785)
 --> Weaponry (Experimental) (cos_norm=0.713, rating=0.900, recency=0.973, score=0.776)
 --> Dying Light: Ox Warrior Bundle (cos_norm=0.696, rating=0.963, recency=0.911, score=0.771)
 --> Break-a-Bot (cos_norm=0.678, rating=0.944, recency=0.989, score=0.763)


[(np.int64(9122),
  np.float32(0.7944304),
  np.float64(0.9804849202),
  np.float64(0.9606749962666136),
  np.float64(0.8482657462360889)),
 (np.int64(8267),
  np.float32(0.7648356),
  np.float64(0.7692307692),
  np.float64(0.9552989198068594),
  np.float64(0.7847609392380444)),
 (np.int64(3109),
  np.float32(0.71286964),
  np.float64(0.9),
  np.float64(0.9733187316441834),
  np.float64(0.7763406181194813)),
 (np.int64(293),
  np.float32(0.69578016),
  np.float64(0.9628975265),
  np.float64(0.9111951814425805),
  np.float64(0.7707451161929)),
 (np.int64(2243),
  np.float32(0.6782502),
  np.float64(0.9444444444000001),
  np.float64(0.9885509482801533),
  np.float64(0.7625191192251357))]